#Profiling
Sometimes computing the likelihood is not as fast as we would like. Theano provides handy profiling tools, which pymc3 provides a wrapper `model.profile` which returns a `ProfileStats` object. Here we'll profile the likelihood and gradient for the stochastic volatility example.

First we build the model.

In [1]:
import numpy as np
from pymc3 import *
from pymc3.math import exp
from pymc3.distributions.timeseries import *

n = 400
returns = np.genfromtxt(get_data_file('pymc3.examples', "data/SP500.csv"))[-n:]
with Model() as model:
    sigma = Exponential('sigma', 1. / .02, testval=.1)
    nu = Exponential('nu', 1. / 10)
    s = GaussianRandomWalk('s', sigma ** -2, shape=n)
    r = StudentT('r', nu, lam=exp(-2 * s), observed=returns)
    


WARNING (theano.configdefaults): Only clang++ is supported. With g++, we end up with strange g++/OSX bugs.
Applied log-transform to sigma and added transformed sigma_log_ to model.
Applied log-transform to nu and added transformed nu_log_ to model.


Then call profile and summarize it.

In [2]:
model.profile(model.logpt).summary()


Function profiling
  Message: /Users/colin/projects/pymc3/pymc3/model.py:349
  Time in 1000 calls to Function.__call__: 4.690456e-02s
  Time in Function.fn.__call__: 2.515221e-02s (53.624%)
  Time in thunks: 1.793075e-02s (38.228%)
  Total compile time: 5.913320e-01s
    Number of Apply nodes: 26
    Theano Optimizer time: 4.063430e-01s
       Theano validate time: 1.725912e-03s
    Theano Linker time (includes C, CUDA code generation/compiling): 1.008379e-01s
       Import time 7.905388e-02s
       Node make_thunk time 9.964800e-02s

Time in all call to theano.grad() 3.328490e-02s
Time since theano import 41.720s
Class
---
<% time> <sum %> <apply time> <time per call> <type> <#call> <#apply> <Class name>
  82.9%    82.9%       0.015s       1.24e-06s     C    12000      12   theano.tensor.elemwise.Elemwise
   5.6%    88.5%       0.001s       5.03e-07s     C     2000       2   theano.tensor.elemwise.Sum
   3.8%    92.3%       0.001s       1.72e-07s     C     4000       4   theano.tensor

In [3]:
model.profile(gradient(model.logpt, model.vars)).summary()

Function profiling
  Message: /Users/colin/projects/pymc3/pymc3/model.py:349
  Time in 1000 calls to Function.__call__: 8.100653e-02s
  Time in Function.fn.__call__: 5.562019e-02s (68.661%)
  Time in thunks: 3.688788e-02s (45.537%)
  Total compile time: 9.140079e-01s
    Number of Apply nodes: 63
    Theano Optimizer time: 5.787621e-01s
       Theano validate time: 8.579969e-03s
    Theano Linker time (includes C, CUDA code generation/compiling): 2.355778e-01s
       Import time 1.562703e-01s
       Node make_thunk time 2.326310e-01s

Time in all call to theano.grad() 1.042541e+00s
Time since theano import 43.896s
Class
---
<% time> <sum %> <apply time> <time per call> <type> <#call> <#apply> <Class name>
  59.3%    59.3%       0.022s       8.41e-07s     C    26000      26   theano.tensor.elemwise.Elemwise
   9.8%    69.1%       0.004s       5.19e-07s     C     7000       7   theano.tensor.elemwise.Sum
   8.8%    77.9%       0.003s       1.62e-06s     C     2000       2   theano.tensor